# 文本特征提取

## 1. 简单的情感分析

In [1]:
# 简单的例子

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.classify import NaiveBayesClassifier

text1 = 'I like the movie so much!'
text2 = 'That is a good movie.'
text3 = 'This is a great one.'
text4 = 'That is a really bad movie.'
text5 = 'This is a terrible movie.'

def proc_text(text):
    """
        预处处理文本
    """
    # 分词
    raw_words = nltk.word_tokenize(text)
    
    # 词形归一化
    wordnet_lematizer = WordNetLemmatizer()    
    words = [wordnet_lematizer.lemmatize(raw_word) for raw_word in raw_words]
    
    # 去除停用词
    filtered_words = [word for word in words if word not in stopwords.words('english')]
    
    # True 表示该词在文本中，为了使用nltk中的分类器
    return {word: True for word in filtered_words}

# 构造训练样本
train_data = [[proc_text(text1), 1],
              [proc_text(text2), 1],
              [proc_text(text3), 1],
              [proc_text(text4), 0],
              [proc_text(text5), 0]]

print(train_data)

[[{'I': True, 'like': True, 'movie': True, 'much': True, '!': True}, 1], [{'That': True, 'good': True, 'movie': True, '.': True}, 1], [{'This': True, 'great': True, 'one': True, '.': True}, 1], [{'That': True, 'really': True, 'bad': True, 'movie': True, '.': True}, 0], [{'This': True, 'terrible': True, 'movie': True, '.': True}, 0]]


In [3]:
# 训练模型
nb_model = NaiveBayesClassifier.train(train_data)

# 测试模型
text6 = 'That is a not bad one.'
print(nb_model.classify(proc_text(text6)))

0


## 2. 词袋模型

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
import os
import re
import jieba.posseg as pseg

### 2.1 文本预处理

In [3]:
# 加载停用词表
stop_words_path = './stop_words/'

stopwords1 = [line.rstrip() for line in open(os.path.join(stop_words_path, '中文停用词库.txt'), 'r',
                                             encoding='utf-8')]
stopwords2 = [line.rstrip() for line in open(os.path.join(stop_words_path, '哈工大停用词表.txt'), 'r',
                                             encoding='utf-8')]
stopwords3 = [line.rstrip() for line in
              open(os.path.join(stop_words_path, '四川大学机器智能实验室停用词库.txt'), 'r', encoding='utf-8')]
stopwords = stopwords1 + stopwords2 + stopwords3

In [4]:
def proc_text(raw_line):
    """
        处理文本数据
        返回分词结果
    """

    # 1. 使用正则表达式去除非中文字符
    filter_pattern = re.compile('[^\u4E00-\u9FD5]+')
    chinese_only = filter_pattern.sub('', raw_line)

    # 2. 结巴分词+词性标注
    word_list = pseg.cut(chinese_only)

    # 3. 去除停用词，保留有意义的词性
    # 动词，形容词，副词
    used_flags = ['v', 'a', 'ad', 'n']
    meaninful_words = []
    for word, flag in word_list:
        if (word not in stopwords) and (flag in used_flags):
            meaninful_words.append(word)
    return ' '.join(meaninful_words)

### 2.2 处理训练数据

In [5]:
ch_text1 = '5月9日，新华社发文称，五一前后住建部约谈了成都、太原等12个城市，主要集中在东北、中西部地区以及政策利好的海南。'
ch_text2 = '响应国家号召，被约谈城市纷纷出台举措应对楼市过热问题。'
ch_text3 = '成都由此将限购对象从自然人升级为家庭，西安对碧桂园等42家房企进行约谈，昆明、贵阳、长春则升级了限售政策。'
ch_text4 = '5月份楼市调控的高潮出现在19日，住建部再次重申坚持房地产调控目标不动摇、力度不放松，并从住房发展规划、住房和土地供应、资金管控、市场监管、落实主体责任等方面做出了明确要求。'
ch_text5 = '调控的目的只有一个，那就是稳定楼市。'

ch_texts = [ch_text1, ch_text2, ch_text3, ch_text4, ch_text5]

In [6]:
corpus = [proc_text(ch_text) for ch_text in ch_texts]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Robin\AppData\Local\Temp\jieba.cache
Loading model cost 0.756 seconds.
Prefix dict has been built succesfully.


In [7]:
corpus

['发文 称 住 建部 谈 集中 地区 政策 利好',
 '响应 国家 号召 谈 出台 举措 应对 楼市 过热 问题',
 '限购 对象 家庭 等家 进行 谈 限售 政策',
 '月份 楼市 出现 住 建部 重申 坚持 目标 动摇 力度 放松 住房 规划 住房 土地 资金 市场监管 落实 主体 责任 方面 做出',
 '目的 稳定 楼市']

### 2.3 建立BoW模型

In [8]:
count_vectorizer = CountVectorizer(max_features=10)
count_vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [9]:
X = count_vectorizer.fit_transform(corpus)

In [10]:
X

<5x10 sparse matrix of type '<class 'numpy.int64'>'
	with 14 stored elements in Compressed Sparse Row format>

In [11]:
# 查看词典
count_vectorizer.vocabulary_

{'建部': 1,
 '政策': 2,
 '楼市': 5,
 '等家': 9,
 '月份': 4,
 '目标': 6,
 '住房': 0,
 '方面': 3,
 '目的': 7,
 '稳定': 8}

In [12]:
X.toarray()

array([[0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 1],
       [2, 1, 0, 1, 1, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 1, 1, 0]], dtype=int64)

### 2.4 处理新文本

In [13]:
new_text = '''面对调控与市场的双面性，人们最关心的是如何做出购房选择。预判楼市如同看待股市，总有两种不同声音。

　　一种声音认为，楼市要涨，再不上车就“晚”了；一种声音认为，楼市求稳，要学会慢慢的“飞”。

　　面对两种声音，较真儿的人总喜欢论出个是非胜负，坦然的人更愿意踏实努力尽力而为。

　　回头看，刚需的人总要上车，被炒的房终会入市。

　　十九大报告已经明确“要加快建立多主体供应、多渠道保障，租购并举的住房制度”。'''
new_pro_text = proc_text(new_text)
new_pro_text

'面对 市场 双面 性 关心 做出 购房 选择 楼市 看待 股市 声音 声音 认为 楼市 要涨 上车 声音 认为 楼市 求稳 学会 飞 面对 声音 喜欢 论出 是非 胜负 坦然 愿意 踏实 努力 回头 需 上车 炒 房 终会 报告 明确 加快 建立 主体 保障 租 购并 举 住房 制度'

In [14]:
count_vectorizer.transform([new_pro_text]).toarray()

array([[1, 0, 0, 0, 0, 3, 0, 0, 0, 0]], dtype=int64)

## 3. 文本分类及TF-IDF

### 3.1 NLTK中的TF-IDF

In [15]:
from nltk.text import TextCollection

text1 = 'I like the movie so much '
text2 = 'That is a good movie '
text3 = 'This is a great one '
text4 = 'That is a really bad movie '
text5 = 'This is a terrible movie'

# 构建TextCollection对象
tc = TextCollection([text1, text2, text3, 
                        text4, text5])
new_text = 'That one is a good movie. This is so good!'
word = 'That'
tf_idf_val = tc.tf_idf(word, new_text)
print('{}的TF-IDF值为：{}'.format(word, tf_idf_val))

That的TF-IDF值为：0.02181644599700369


### 3.2 sklearn中的TF-IDF

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
feat = vectorizer.fit_transform([text1, text2, text3, text4, text5])

In [12]:
feat.toarray()

array([[0.        , 0.        , 0.        , 0.        , 0.48127008,
        0.27113917, 0.48127008, 0.        , 0.        , 0.48127008,
        0.        , 0.        , 0.48127008, 0.        ],
       [0.        , 0.6614376 , 0.        , 0.3726424 , 0.        ,
        0.3726424 , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.53364369, 0.        , 0.        ],
       [0.        , 0.        , 0.58042343, 0.32700044, 0.        ,
        0.        , 0.        , 0.58042343, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.46828197],
       [0.55167715, 0.        , 0.        , 0.31080528, 0.        ,
        0.31080528, 0.        , 0.        , 0.55167715, 0.        ,
        0.        , 0.44508965, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.3726424 , 0.        ,
        0.3726424 , 0.        , 0.        , 0.        , 0.        ,
        0.6614376 , 0.        , 0.        , 0.53364369]])

In [17]:
vectorizer.get_feature_names()

['bad',
 'good',
 'great',
 'is',
 'like',
 'movie',
 'much',
 'one',
 'really',
 'so',
 'terrible',
 'that',
 'the',
 'this']

In [18]:
feat_array = feat.toarray()
feat_array.shape

(5, 14)

In [19]:
feat_array[0:2, :]

array([[0.        , 0.        , 0.        , 0.        , 0.48127008,
        0.27113917, 0.48127008, 0.        , 0.        , 0.48127008,
        0.        , 0.        , 0.48127008, 0.        ],
       [0.        , 0.6614376 , 0.        , 0.3726424 , 0.        ,
        0.3726424 , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.53364369, 0.        , 0.        ]])

In [20]:
vectorizer.transform([new_text]).toarray()

array([[0.        , 0.67082255, 0.        , 0.37792972, 0.        ,
        0.18896486, 0.        , 0.33541128, 0.        , 0.33541128,
        0.        , 0.27060771, 0.        , 0.27060771]])

### 3.3 中文TF-IDF

In [1]:
import os
import re
import jieba.posseg as pseg

ch_text1 = '5月9日，新华社发文称，五一前后住建部约谈了成都、太原等12个城市，主要集中在东北、中西部地区以及政策利好的海南。'
ch_text2 = '响应国家号召，被约谈城市纷纷出台举措应对楼市过热问题。'
ch_text3 = '成都由此将限购对象从自然人升级为家庭，西安对碧桂园等42家房企进行约谈，昆明、贵阳、长春则升级了限售政策。'
ch_text4 = '5月份楼市调控的高潮出现在19日，住建部再次重申坚持房地产调控目标不动摇、力度不放松，并从住房发展规划、住房和土地供应、资金管控、市场监管、落实主体责任等方面做出了明确要求。'
ch_text5 = '调控的目的只有一个，那就是稳定楼市。'

ch_texts = [ch_text1, ch_text2, ch_text3, ch_text4, ch_text5]

In [2]:
# 加载停用词表
stop_words_path = './stop_words/'

stopwords1 = [line.rstrip() for line in open(os.path.join(stop_words_path, '中文停用词库.txt'), 'r',
                                             encoding='utf-8')]
stopwords2 = [line.rstrip() for line in open(os.path.join(stop_words_path, '哈工大停用词表.txt'), 'r',
                                             encoding='utf-8')]
stopwords3 = [line.rstrip() for line in
              open(os.path.join(stop_words_path, '四川大学机器智能实验室停用词库.txt'), 'r', encoding='utf-8')]
stopwords = stopwords1 + stopwords2 + stopwords3

In [18]:
len(stopwords)

2489

In [15]:
def proc_text(raw_line):
    """
        处理文本数据
        返回分词结果
    """

    # 1. 使用正则表达式去除非中文字符
    filter_pattern = re.compile('[^\u4E00-\u9FD5]+')
    chinese_only = filter_pattern.sub('', raw_line)

    # 2. 结巴分词+词性标注
    word_list = pseg.cut(chinese_only)

    # 3. 去除停用词，保留有意义的词性
    # 动词，形容词，副词
    used_flags = ['v', 'a', 'ad']
    meaninful_words = []
    for word, flag in word_list:
#         if (word not in stopwords) and (flag in used_flags):
        if word not in stopwords:
            meaninful_words.append(word)
    return ' '.join(meaninful_words)

In [16]:
corpus = [proc_text(ch_text) for ch_text in ch_texts]

In [17]:
corpus

['月 日 新华社 发文 称 五一 住 建部 约 谈 成都 太原 城市 主要 集中 东北 中西部 地区 政策 利好 海南',
 '响应 国家 号召 约 谈 城市 纷纷 出台 举措 应对 楼市 过热 问题',
 '成都 限购 对象 自然人 升级 家庭 西安 碧桂园 等家 房企 进行 约 谈 昆明 贵阳 长春 升级 限售 政策',
 '月份 楼市 调控 高潮 出现 日 住 建部 重申 坚持 房地产 调控 目标 动摇 力度 放松 住房 发展 规划 住房 土地 供应 资金 管控 市场监管 落实 主体 责任 方面 做出 明确要求',
 '调控 目的 稳定 楼市']

In [18]:
ch_vectorizer = TfidfVectorizer()
ch_feats = ch_vectorizer.fit_transform(corpus)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [19]:
ch_vectorizer.get_feature_names()

['东北',
 '中西部',
 '主体',
 '主要',
 '举措',
 '五一',
 '住房',
 '供应',
 '做出',
 '出台',
 '出现',
 '利好',
 '力度',
 '动摇',
 '升级',
 '发展',
 '发文',
 '号召',
 '响应',
 '国家',
 '土地',
 '地区',
 '坚持',
 '城市',
 '太原',
 '家庭',
 '对象',
 '市场监管',
 '应对',
 '建部',
 '成都',
 '房企',
 '房地产',
 '放松',
 '政策',
 '新华社',
 '方面',
 '昆明',
 '明确要求',
 '月份',
 '楼市',
 '海南',
 '目标',
 '目的',
 '碧桂园',
 '稳定',
 '等家',
 '管控',
 '纷纷',
 '自然人',
 '落实',
 '西安',
 '规划',
 '调控',
 '责任',
 '贵阳',
 '资金',
 '过热',
 '进行',
 '重申',
 '长春',
 '问题',
 '限售',
 '限购',
 '集中',
 '高潮']

In [20]:
ch_feats.toarray()[0, :]

array([0.27112655, 0.27112655, 0.        , 0.27112655, 0.        ,
       0.27112655, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.27112655, 0.        , 0.        , 0.        ,
       0.        , 0.27112655, 0.        , 0.        , 0.        ,
       0.        , 0.27112655, 0.        , 0.21874319, 0.27112655,
       0.        , 0.        , 0.        , 0.        , 0.21874319,
       0.21874319, 0.        , 0.        , 0.        , 0.21874319,
       0.27112655, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.27112655, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.27112655,
       0.        ])